In [4]:
import os
import sys
import argparse 
import math
import time
import h5py
import joblib
import subprocess
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt 
import seaborn as sns

In [6]:
os.chdir('/Users/pengl7/Downloads/WGS/compare-variants/NIST/')

In [7]:
%ls -lth

total 114176
-rw-r--r--  1 pengl7  NIH\Domain Users    22M Sep 28 11:51 long_cleared_QD.csv
-rw-r--r--  1 pengl7  NIH\Domain Users    21M Sep 25 16:58 long_cleared.csv
-rw-r-----  1 pengl7  NIH\Domain Users    12M Sep 25 10:35 long4_with_title


In [8]:
# df = pd.read_csv("long4_with_title", sep="\t",index_col="POS", na_values={}, dtype={'UNMATCH': "category", "DP": "float", "GQ": "float", "MQ": "float"}, )
df = pd.read_csv("long4_with_title", sep="\t")

/Users/pengl7/opt/anaconda3/envs/genomics/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(df.shape)
print(df.columns.to_list())

(339882, 9)
['POS', 'UNMATCH', 'TYPE', 'QUAL', 'FS', 'AF', 'DP', 'GQ', 'MQ']


In [6]:
df.dtypes

POS          int64
UNMATCH      int64
TYPE         int64
QUAL       float64
FS         float64
AF         float64
DP          object
GQ          object
MQ          object
dtype: object

## Get rid of some non-numeric values and change data types

In [11]:
df[df["MQ"]== "."]

,POS,UNMATCH,TYPE,QUAL,FS,AF,DP,GQ,MQ
290425,219791127,1,1,99.14,0.0,1.0,0,8,.
312958,234937551,1,1,53.70,0.0,0.5,0,6,.


In [13]:
myFilter = (df["DP"]!= ".") & (df["GQ"]!= ".") & (df["MQ"]!= ".")
df = df[myFilter]
print(df.shape)

(339860, 9)


In [16]:
# change data type 
cols = ['DP', 'GQ', 'MQ']
for col in cols:
    df[col]= df[col].astype("float")

In [37]:
df.to_csv("long_cleared.csv")

## Apply logsistic regression model

In [17]:
df.describe()

,POS,UNMATCH,TYPE,QUAL,FS,AF,DP,GQ,MQ
count,3.398600e+05,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000
mean,1.256499e+08,0.102457,0.161160,480.445892,1.733951,0.701240,19.790970,78.836871,59.430544
std,7.641397e+07,0.303249,0.367679,720.769333,3.342236,0.247945,22.881138,27.074520,2.707473
min,1.014600e+04,0.000000,0.000000,30.250000,0.000000,0.000200,0.000000,0.000000,20.000000
25%,5.925370e+07,0.000000,0.000000,233.770000,0.000000,0.500000,14.000000,59.000000,60.000000
50%,1.148291e+08,0.000000,0.000000,385.770000,0.000000,0.500000,19.000000,99.000000,60.000000
75%,1.950171e+08,0.000000,0.000000,647.770000,2.218000,1.000000,24.000000,99.000000,60.000000
max,2.489456e+08,1.000000,1.000000,117461.000000,357.189000,1.000000,3142.000000,99.000000,60.000000


In [19]:
this_formula = "UNMATCH ~ TYPE + DP + GQ + QUAL + FS + AF + MQ"
res = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit() 
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               339860
Model:                            GLM   Df Residuals:                   339852
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -86131.
Date:                Fri, 25 Sep 2020   Deviance:                   1.7226e+05
Time:                        16:28:26   Pearson chi2:                 3.59e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.9337      0.122    114.109      0.000      13.694      14.173
TYPE           1.7849      0.014    128.356      0.000       1.758       1.812
DP             0.0154      0.001     17.306      0.000       0.014       0.017
GQ            -0.0250      0.000    -83.688      0.000      -0.026      -0.024
QUAL          -0.0004   2.74e-05    -16.150      0.000      -0.000      -0.000
FS             0.0134      0.002      6.387      0.000       0.009       0.018
AF            -1.9726      0.041    -47.853      0.000      -2.053      -1.892
MQ            -0.2293      0.002   -116.589      0.000      -0.233      -0.225
==============================================================================
"""

In [23]:
df["QualByDepth"] = df["GQ"]/(df["DP"]+1)

In [24]:

formula2 = "UNMATCH ~ QualByDepth + TYPE + DP + GQ + QUAL + FS + AF + MQ"
res = sm.formula.glm(formula=formula2, family=sm.families.Binomial(), data=df).fit() 
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               339860
Model:                            GLM   Df Residuals:                   339851
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 25 Sep 2020   Deviance:                          nan
Time:                        16:33:00   Pearson chi2:                 5.36e+15
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      11.3704      0.129     88.282      0.000      11.118      11.623
QualByDepth     0.3220      0.005     66.974      0.000       0.313       0.331
TYPE            1.6021      0.014    111.655      0.000       1.574       1.630
DP              0.0660      0.002     40.439      0.000       0.063       0.069
GQ             -0.0414      0.000    -96.257      0.000      -0.042      -0.041
QUAL           -0.0013   3.88e-05    -33.082      0.000      -0.001      -0.001
FS              0.0158      0.002      7.287      0.000       0.012       0.020
AF             -1.0784      0.046    -23.495      0.000      -1.168      -0.988
MQ             -0.2081      0.002   -103.908      0.000      -0.212      -0.204
===============================================================================
"""

In [26]:
print("Coefficeients")
print(res.params)
print()
print("p-Values")
print(res.pvalues)
print()
print("Dependent variables")
print(res.model.endog_names)

Coefficeients
Intercept      11.370424
QualByDepth     0.321993
TYPE            1.602123
DP              0.065982
GQ             -0.041431
QUAL           -0.001284
FS              0.015807
AF             -1.078384
MQ             -0.208142
dtype: float64

p-Values
Intercept       0.000000e+00
QualByDepth     0.000000e+00
TYPE            0.000000e+00
DP              0.000000e+00
GQ              0.000000e+00
QUAL           5.310002e-240
FS              3.162637e-13
AF             4.538994e-122
MQ              0.000000e+00
dtype: float64

Dependent variables
UNMATCH
